In [1]:
from models import Generator

In [8]:
gen = Generator(3, 3, use_bn=True)

In [3]:
import torch
gen(torch.FloatTensor(3, 3, 128, 128).random_(0, 1)).shape

torch.Size([3, 3, 128, 128])

In [10]:
gen.eval()
gen.fuse()

In [11]:
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import math
import cv2
from time import perf_counter
import os
import yaml
#from fid_score import calculate_fid_given_paths
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from torch.quantization.default_mappings import DEFAULT_MODULE_MAPPING
from torch.quantization.observer import *
from torch.quantization.fake_quantize import *
from torch.quantization.qconfig import *
from torch.quantization import prepare_qat
from torch.quantization import prepare

def calibrate_model(gen, num_forwards=100):
    for i in range(num_forwards):
        torch.manual_seed(i)
        inpGen = torch.FloatTensor(8, 3, 128, 128).random_(0, 1).to(device='cpu')
        with torch.no_grad():
            gen.eval()
            fake_imgs = gen(inpGen)
    return gen

weight_quant = MovingAveragePerChannelMinMaxObserver.with_args(
                                                           dtype=torch.qint8,
                                                           qscheme=torch.per_channel_symmetric,
                                                           reduce_range=True)

qconfig = QConfig(activation=HistogramObserver.with_args(reduce_range=True),
                      weight=weight_quant)
gen.qconfig = qconfig
prepare(gen, inplace=True)
gen = calibrate_model(gen, 50)

torch.quantization.convert(gen.cpu().eval(), inplace=True)
#print('The model created')
gen.apply(torch.quantization.disable_observer)

Generator(
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model): Sequential(
    (0): ConvNormReLU(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(1, 1), scale=1.0, zero_point=0, padding=(3, 3))
      (1): Identity()
      (2): Identity()
    )
    (1): ConvNormReLU(
      (0): QuantizedConvReLU2d(64, 128, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0, padding=(1, 1))
      (1): Identity()
      (2): Identity()
    )
    (2): ConvNormReLU(
      (0): QuantizedConvReLU2d(128, 256, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0, padding=(1, 1))
      (1): Identity()
      (2): Identity()
    )
    (3): ResidualBlock(
      (conv_block): Sequential(
        (0): ConvNormReLU(
          (0): QuantizedConvReLU2d(256, 256, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
          (1): Identity()
          (2): Identity()
        )
        (1): ConvNorm(
 

In [7]:
%%time
gen(torch.FloatTensor(64, 3, 128, 128).random_(0, 1))

CPU times: user 12.3 s, sys: 119 ms, total: 12.4 s
Wall time: 1.62 s


tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156],
          [0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156],
          [0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156],
          ...,
          [0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156],
          [0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156],
          [0.0156, 0.0156, 0.0156,  ..., 0.0156, 0.0156, 0.0156]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [9]:
%%time
gen(torch.FloatTensor(64, 3, 128, 128).random_(0, 1))

CPU times: user 22.3 s, sys: 2.44 s, total: 24.8 s
Wall time: 3.15 s


tensor([[[[-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027],
          [-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027],
          [-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027],
          ...,
          [-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027],
          [-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027],
          [-0.0027, -0.0027, -0.0027,  ..., -0.0027, -0.0027, -0.0027]],

         [[ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155],
          [ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155],
          [ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155],
          ...,
          [ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155],
          [ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155],
          [ 0.0155,  0.0155,  0.0155,  ...,  0.0155,  0.0155,  0.0155]],

         [[-0.0056, -0.0056, -0.0056,  ..., -0.0056, -0.0056, -0.0056],
          [-0.0056, -0.0056, -